## Import Packages

In [1]:
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
import sqlite3
import numpy as np
from datetime import datetime

C:\Users\yoche\AppData\Local\Temp\ipykernel_14540\3831387850.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class Datetimeclass:
    def __init__(self) -> None:
        self.now = datetime.now()
        pass
    def today_format(self,format='%m%d'):
        return self.now.strftime(format)


## Crawlering & Storage

In [3]:
class DatabaseHandler:
    def __init__(self, db_name):
        self.db_file = db_name
        self.conn = None

    def create_connection(self):
        try:
            self.conn = sqlite3.connect(self.db_file)
        except:
            print("Connection error")

    def create_table(self, create_table_query):
        try:
            c = self.conn.cursor()
            c.execute(create_table_query)
            # self.conn.commit()  # Uncomment this line if you want to commit changes immediately
        except:
            print("Create table error")

    def add_new_row(self, insert_list):
        # Insert new data
        try:
            cursor = self.conn.cursor()
            str_insert_list = "'" + "','".join(insert_list) + "'"
            add_new_row_query = f"""INSERT INTO PTT(board, id, pushes, author, title)
                                   VALUES({str_insert_list})"""

            cursor.execute(add_new_row_query)
            self.conn.commit()
        except:
            print('?')
            print(insert_list)
    
    def get_data(self,query):
        query_result = pd.read_sql(query,self.conn)
        return query_result

    def close_connection(self):
        if self.conn:
            self.conn.close()



database_name = f"mydatabase_{Datetimeclass().today_format()}.db"

sql_create_ptt_table_query = f"""CREATE TABLE IF NOT EXISTS PTT (
                                    board text NOT NULL,
                                    id text,
                                    pushes text,
                                    author text,
                                    title text
                                );"""


# new_data = ["board_value", "id_value", "pushes_value", "author_value", "title_value"]
# handler.add_new_row(new_data)


In [4]:
if __name__ == '__main__':

    handler = DatabaseHandler(database_name)

    # create a database connection
    handler.create_connection()

    # create tables
    if handler.conn is not None:
        handler.create_table(sql_create_ptt_table_query)
    else:
        print("Error! cannot create the database connection.")

    # 用openpyxl寫進worksheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append(["看板名稱",'id',"推文數","文章作者","文章名稱"]) 
    
    for board_name in ('NBA',"Stock",'Boy-Girl','Baseball',"Lifeismoney"):
        url = 'https://www.ptt.cc/bbs/'+board_name+'/index.html'
        for page in range(1,2): #30頁
            print(url)  #當前的url
            print(board_name,"版page",page)
            time.sleep(2)
            web = requests.get(url,cookies={'over18':'1'})
            soup = BeautifulSoup(web.text, "html.parser")
            pushes = soup.find_all('div',class_='nrec')
            authors = soup.find_all('div',class_='author')
            titles = soup.find_all('div', class_='title')     # 取得 class 為 title 的 div 內容
            for i,j,k in zip(titles,pushes,authors):
                if i.find('a') != None:                         # 判斷如果不為 None
                    a = j.text
                    #print(a,k.text,end=" ")        #推文數,作者
                    #print(i.find('a').get_text())                 # 取得 div 裡 a 的內容，使用 get_text() 取得文字
                    
                    id = i.find('a').get("href")[i.find('a').get("href").find(board_name):i.find('a').get("href").find('.A.')]
                    
                    b=0
                    
                    if a =='爆':
                        url_article = "https://www.ptt.cc"+i.find('a').get('href')
                        web_article = requests.get(url_article,cookies={'over18':'1'})
                        soup_article = BeautifulSoup(web_article.text, "html.parser")
                        b=0
                        pushnumber = soup_article.find_all('div',class_='push')
                        for m in pushnumber:
                            if m.text[0] == '推':
                                b+=1
                            elif m.text[0] == '噓':
                                b-=1
                        #print("爆",b)
                    #print([board_name,a,k.text,i.text])
                    
                    if b>0:
                        #print([board_name,str(b),k.text,i.text])
                        data = [board_name,id,str(b),k.text,i.text]
                        ws.append(data)
                        # add_new_row(data)
                        handler.add_new_row(data)
                    else:
                        #print([board_name,a,k.text,i.text])
                        data = [board_name,id,a,k.text,i.text]
                        ws.append(data)
                        # add_new_row(data)
                        handler.add_new_row(data)

                    

            url = "https://www.ptt.cc"+soup.find_all("a",class_='btn wide')[1].get('href') #上一頁的href


    # Close the connection when done
    handler.close_connection()
    wb.save("PPT1.xlsx")


https://www.ptt.cc/bbs/NBA/index.html
NBA 版page 1
https://www.ptt.cc/bbs/Stock/index.html
Stock 版page 1
https://www.ptt.cc/bbs/Boy-Girl/index.html
Boy-Girl 版page 1
https://www.ptt.cc/bbs/Baseball/index.html
Baseball 版page 1
https://www.ptt.cc/bbs/Lifeismoney/index.html
Lifeismoney 版page 1


In [9]:
query = """
            SELECT  name
            FROM sqlite_schema
            WHERE type ='table' AND 
                  name NOT LIKE 'sqlite_%';
        """
pd.read_sql(query,sqlite3.connect('mydatabase_0205.db'))

,name
0,PTT


In [13]:
query = """SELECT * FROM PTT"""
pd.read_sql(query,sqlite3.connect('mydatabase_0205.db'))

,board,id,pushes,author,title
0,NBA,NBA/M.1707091070,102,thnlkj0665,\n[情報] Ben Simmons 將缺席明天面對勇士隊比賽\n
1,NBA,NBA/M.1707093002,62,laptic,\n[Live] 公鹿 @ 爵士\n
2,NBA,NBA/M.1707094813,99,laptic,\n[Live] 拓荒者 @ 金塊\n
3,NBA,NBA/M.1707095045,164,EZ78,\n[情報] Joel Embiid動刀\n
4,NBA,NBA/M.1707096285,5,EZ78,\n[公告] 板主投票開始\n
...,...,...,...,...,...
68,Lifeismoney,Lifeismoney/M.1707103183,,pp18637,\n[情報] 大魯閣用icashPay滿$500贈10%op\n
69,Lifeismoney,Lifeismoney/M.1704123061,66,AllenSue,\n[情報] === 元月全台捐血贈品 === (1/26更新)\n
70,Lifeismoney,Lifeismoney/M.1705326040,,finhisky,\n[公告] 板規(113.1.15修訂)暨違規公告區\n
71,Lifeismoney,Lifeismoney/M.1692033175,59,finhisky,\n[公告] 贈送集中文\n
